In [3]:
from neo4j import GraphDatabase
import json

In [8]:
with open('credentials.json') as json_file:
    credentials = json.load(json_file)

username = credentials['username']
pwd = credentials['password']

In [10]:
# BEFORE running this, still need to run `bin\neo4j console` to enable bolt on 127.0.0.1:7687

driver = GraphDatabase.driver("bolt://localhost:7687", auth=(username, pwd))

In [ ]:
# hellow world example: https://github.com/neo4j/neo4j-python-driver
# driver manual: https://neo4j.com/docs/driver-manual/1.7/get-started/